# Listing of vehicle counting points

In [1]:
# Import of needed libraries

# Libraries for retieving data from the GraphQL API
import urllib3
from gql import gql, Client
from gql.transport.requests import RequestsHTTPTransport

# Imports for plotting
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
%matplotlib inline

# Magic stuff from the Pandas people, flatteing the JSON files into something that can pass off as a table
from pandas.io.json import json_normalize

# Tools to work with dates
from datetime import datetime
from dateutil.relativedelta import relativedelta

# Remove warnings related to HTTPS, no need as data is public
urllib3.disable_warnings()

# And finaly the Pandas library for working with the data 
import pandas as pd

In [2]:
# Setup of the GQL client to talk to the NPRA Datainn API

sample_transport=RequestsHTTPTransport(
    url='https://www.vegvesen.no/trafikkdata/api/',
    use_json=True,
    headers={
        "Content-type": "application/json",
    },
    verify=False
)

client = Client(
    retries=3,
    transport=sample_transport,
    fetch_schema_from_transport=True,
)

In [3]:
query = gql('''{
  trafficRegistrationPoints(searchQuery: {roadCategoryIds: [E, R]}) {
    id
    name
    location {
      roadReference { shortForm
      roadCategory {name}
      }
      coordinates {
        latLon {
          lat
          lon
        }
      }
    }
  }
}
''')

data = client.execute(query)

#data

traffic_registration_points = data['trafficRegistrationPoints']

rows = []

for trp in traffic_registration_points:
    row = [trp['id'], trp['name'], trp['location']['coordinates']['latLon']['lat'],
           trp['location']['coordinates']['latLon']['lon'],trp['location']['roadReference']['shortForm'],
          trp['location']['roadReference']['roadCategory']['name']]
    rows.append(row)
    
    
    


df = pd.DataFrame(rows)
column_names = ['id', 'name', 'lat', 'lon', 'road_reference','road_category']
df.columns=column_names
df.head()

,id,name,lat,lon,road_reference,road_category
0,56100V804816,RAMPE SØRÅS MOT HOP,60.325803,5.337152,EV39 S78D1 m6434 KD3 m172,Europaveg
1,03486V319647,ØLEN/ETNE,59.623114,5.886108,EV134 S4D1 m17825,Europaveg
2,20318V625294,FESTNINGTUNNEL,59.911180,10.724291,EV18 S55D1 m5076,Europaveg
3,00030V705190,MATRAND S,60.020618,12.113421,RV2 S2D1 m4510,Riksveg
4,29614V805708,SOTRABRUA VEST,60.372022,5.155573,RV555 S1D1 m11739,Riksveg


In [9]:
# Making data available into other notebooks
vehicle_cointing_points = df
%store vehicle_cointing_points

Stored 'vehicle_cointing_points' (DataFrame)
